In [ ]:
#@title Init env
%pip install -q git+https://github.com/bigscience-workshop/petals.git transformers 2> log

import torch
from transformers import BloomTokenizerFast 
from petals.client import DistributedBloomForCausalLM
import ipywidgets as widgets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 113.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 110.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 104.8 M

In [ ]:
#@title Load model

MODEL_NAME = "bigscience/bloomz-petals" #@param ["bigscience/bloomz-petals", "bigscience/bloom-petals"]
tokenizer = BloomTokenizerFast.from_pretrained(MODEL_NAME, padding_side='right')
model = DistributedBloomForCausalLM.from_pretrained(MODEL_NAME).cuda()

Downloading:   0%|          | 0.00/319 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.19G [00:00<?, ?B/s]

In [ ]:
#@title Helper functions

import time

def PrintContinuation(prompt, local_model, single_hook=None, batch=1, limit_tokens = 50):
    global input_dict, outputs

    past_key_values = None  # used to keep track of conversation history
    token_ids = tokenizer(prompt, return_tensors='pt')["input_ids"]
    token_ids = token_ids[0, -1500:] # limit attention to suffix
    token_ids = torch.stack([token_ids] * batch)
    token_ids = token_ids.to(local_model.device)

    output = [""] * batch
    batch_time = 0
    
    with torch.inference_mode():
        with local_model.transformer.h.inference_session(max_length=token_ids.shape[1] + limit_tokens + 2) as sess:
            pre_tokens = token_ids[:,:-1]
            batch_size = 32
            for i in range(0, pre_tokens.shape[1], batch_size):
                embs = local_model.transformer.word_embeddings(pre_tokens[:,i:i+batch_size])
                embs = local_model.transformer.word_embeddings_layernorm(embs)
                h = sess.step(embs)

            token_ids = token_ids[:,-1:]
            for i in range(limit_tokens + 2):
                if i == 5:
                    start_time = time.perf_counter()

                embs = local_model.transformer.word_embeddings(token_ids)
                embs = local_model.transformer.word_embeddings_layernorm(embs)

                h = sess.step(embs)
                h_last = local_model.transformer.ln_f(h[:, -1])
                last_logits = local_model.lm_head(h_last)

                for j in range(batch):
                    last_logits[j, last_logits[j].topk(k=10).indices] += 10

                token_ids = torch.multinomial(last_logits.softmax(-1), 1)
                output = [stream + tokenizer.decode(ix) for stream, ix in zip(output, token_ids)]

                if single_hook is not None:
                    if not single_hook(tokenizer.decode(token_ids[0])):
                        batch_time = (time.perf_counter() - start_time) / (i - 4)
                        break
                if i == limit_tokens:
                    batch_time = (time.perf_counter() - start_time) / (i - 4)
                    break

    return output, batch_time


def CutSentence(text):
    end = 0
    for punct in ".!?\n":
        end = max(end, text.rfind(punct))
    if end == 0:
        end = len(text) - 1
    return text[:end + 1]

In [ ]:
#@title GM interface
MASTER_PREFIX = "Dungeon Master: "
PLAYER_PREFIX = "John: "

story = f"""It is a fantasy role-play game.

{MASTER_PREFIX}You are John, a wizard living in the kingdom of Larion. You have a staff and a spellbook. You finish your long journey and finally arrive at the ruin you've been looking for. You have come here searching for a mystical spellbook of great power called the book of essence. You look around and see the ancient ruins of an elf tower. The ruins have not been touched for decades. You look at the tower, and you can see a set of stone stairs that seem to lead somewhere deep inside the tower.
{PLAYER_PREFIX}I walk upstairs
{MASTER_PREFIX}You climb up the stairs in the ruined tower. There is a door on the second floor of the tower, the door seems to be made of enchanted wood.
{PLAYER_PREFIX}I ask the door if I may to come in
{MASTER_PREFIX}The door sighs open and you walk into the room."""

def ReprStory(text):
    return text.replace('\n', '<br>')

top_label = widgets.HTML(value=ReprStory(story))
user_option = widgets.Textarea(
    placeholder='What do you do?',
    disabled=False
)

submiter = widgets.Button(
    description="Continue",
    button_style='success'
)

user_inputs = [user_option, submiter]

def DoCuntinue(_):
    global story
    story += "\n" + PLAYER_PREFIX + user_option.value + "\n" + MASTER_PREFIX
    top_label.value = ReprStory(story)

    def uprint(text):
        global story
        story += text
        top_label.value = ReprStory(story)
        return not (PLAYER_PREFIX[:-2] in text)

    submiter.layout.visibility = 'hidden'

    PrintContinuation(story, model, uprint, 1, 50)

    story = CutSentence(story)
    top_label.value = ReprStory(story)
    user_option.value = ""

    submiter.layout.visibility = 'visible'

submiter.on_click(DoCuntinue)

display(widgets.VBox(
    [top_label] + user_inputs
))


In [ ]:
#@title Co-writer interface

PROMPT_CUSTOM = 'I have a better idea'
PROMPT_ROLLBACK = 'Rollback the last move'
prompts = ['Once upon a time',
'This is fantasy story about a wizard living in the kingdom of Larion. He has a staff and a spellbook. He finishes his long journey and finally arrive at the ruin he have been looking for.',
"It is a fantasy role-play game.\n\nDM: Moving southwest through the trees you make good progress and the journey is even quite pleasant. After three days travelling you have stopped for a break when your attention is drawn to an argument concerning some brightly coloured purple fruits growing on nearby trees. Several of your men served in India for a time and declare the fruits to be edible and very tasty, others are more doubtful and believe that the fruits might be poisonous.What do you do?\nPlayer: Ignore the fruit and trek on\nDM: Ignoring the fruit you march on and find that the trees begin to thin out even more by evening before giving way entirely to a gentle plain with only a few trees scattered here and there. Snaking across the horizon from south to north is the blue ribbon of a river, the Frontier River that Donovan's expedition crossed before vanishing. You are wary of going too far south so tomorrow you can march either northwest or west towards the river.Which direction do you travel in?\nPlayer: West towards the river\nDM:",
PROMPT_ROLLBACK,
PROMPT_CUSTOM]

MASTER_PREFIX = "Dungeon Master: "
PLAYER_PREDIX = "John: "

story_log = []


top_label = widgets.HTML(value="Choose adventure")
selector = widgets.RadioButtons(
    options=prompts,
    layout={'width': 'max-content'}
)
user_option = widgets.Textarea(
    placeholder='Your own idea',
    disabled=False
)
submiter = widgets.Button(
    description="Continue",
    button_style='success'
)

user_inputs = [selector, user_option, submiter]

def DoCuntinue(_):
    global selector, story_log
    if selector.value == PROMPT_ROLLBACK:
        story_log = story_log[:-1]
    elif selector.value == PROMPT_CUSTOM:
        story_log.append(user_option.value)
    else:
        story_log.append(selector.value)

    pre_story = '\n'.join(story_log).replace('\n', '<br>')
    top_label.value = pre_story + '<br>'

    def uprint(text):
        top_label.value += text
        return True

    for i in range(len(user_inputs)):
        user_inputs[i].layout.visibility = 'hidden'

    options, _ = PrintContinuation(pre_story, model, uprint, 5, 50)

    options = [CutSentence(o) for o in options]

    top_label.value=pre_story
    options.append(PROMPT_ROLLBACK)
    options.append(PROMPT_CUSTOM)
    selector.options = options
    user_option.value = ""

    for i in range(len(user_inputs)):
        user_inputs[i].layout.visibility = 'visible'

submiter.on_click(DoCuntinue)

display(widgets.VBox(
    [top_label] + user_inputs
))

Jan 27 15:25:06.572 [INFO] Peer 12D3KooWCw5TVPZHBNbcsxdjL5HhVYhQrMB1vYjtUoHU5366Hy1x did not respond, banning it temporarily
Jan 27 15:25:06.574 [WARN] [/usr/local/lib/python3.8/dist-packages/petals/client/routing/sequence_manager.py.rpc_info:264] Caught exception when gathering information from peer 12D3KooWCw5TVPZHBNbcsxdjL5HhVYhQrMB1vYjtUoHU5366Hy1x (retry in 0 sec): P2PDaemonError('routing: not found')
